In [ ]:
import os
import sqlalchemy as sa
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
from dotenv import load_dotenv

# Load the .env configuration
ENV_FILE = '.env'
_ = load_dotenv()

DB_URL = os.getenv('DB_URL')
CSVS = os.getenv('CSV_FOLDER')
IMGS = os.getenv('IMAGE_FOLDER')

# Connection to the db
engine = sa.create_engine(DB_URL)
conn = engine.connect()

In [ ]:
# Get discover/connection ratio
sql_query = """
SELECT
    first_seen,
    origin
FROM enrs
ORDER BY first_seen ASC;
"""

df = pd.read_sql_query(sa.text(sql_query), conn)
display(df)
# Peer Discover Rate
sns.set()
p = sns.lineplot(data=df, x='first_seen', y=np.linspace(0, len(df), len(df)), hue='origin')
p.set(title='Unique discovered nodes by Ragno', xlabel='Crawling dates', ylabel='Aggregated unique nodes')
plt.xticks(rotation=30)
plt.savefig(IMGS+'/node_discovery_ratio.png')


In [ ]:
# Connection ratio
sql_query = """
SELECT
    node_id,
    first_connected
FROM node_info
WHERE first_connected IS NOT NULL
ORDER BY first_connected ASC;
"""

df = pd.read_sql_query(sa.text(sql_query), conn)
display(df)
# Peer Discover Rate
sns.set()
p = sns.lineplot(data=df, x='first_connected', y=np.linspace(0, len(df), len(df)))
p.set(title='Unique connected nodes by Ragno', xlabel='Crawling dates', ylabel='Aggregated unique nodes')
plt.xticks(rotation=30)
plt.savefig(IMGS+'/node_connection_ratio.png')

In [ ]:
# Node uptime distribution (curren non-deprecated node count)
sql_query = """
SELECT
    deprecated,
    count(deprecated) as nodes
FROM node_info
GROUP BY deprecated
ORDER BY nodes DESC;
"""

df = pd.read_sql_query(sa.text(sql_query), conn)
display(df)
# Peer Discover Rate
sns.set()
p = sns.barplot(data=df, x='deprecated', y='nodes')
p.set(title='Ratio of active nodes detected by Ragno', xlabel='deprecation status', ylabel='Unique nodes')
plt.xticks(rotation=30)
plt.savefig(IMGS+'/current_actve_node_count.png')

In [ ]:
 # Get the client Distribution
sql_query = """
SELECT
    client_name,
    count(client_name) as nodes
FROM node_info
WHERE deprecated='false'
GROUP BY client_name
ORDER BY nodes DESC;
"""

df = pd.read_sql_query(sa.text(sql_query), conn)
display(df)